In this script we will extract metrics for each commit. For that you need to download and install Understand Tool, available here https://scitools.com/trial-download-3/ . To run you will need license to Understand, student get free in https://scitools.com/non-commercial-license/

In [1]:
pip install GitPython

Note: you may need to restart the kernel to use updated packages.


In [3]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [4]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil

In [5]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

Scalabrino's Readability

In [6]:
class ScalabrinoReadability(object):
    def __init__(self, project):
#         super(Readability, self).__init__()
        self.project = project
        self.repositoryPath = "repositories\\"+project

    def getReadability(self):
        print("Procurando arquivos")
        javaFiles = findPaths("*.java", self.repositoryPath)
        print(f"{len(javaFiles)} arquivos encontrados")
        if len(javaFiles) == 0: return 0
        readability = 0
        array = []
        for javaFile in javaFiles:
            r = os.popen(f"java -jar rsm.jar {javaFile}").read().replace("\n", "").split("\t")[-1]
            if (not "NaN" in r):
                array.append(float(r))
            else:
                print("Drop NaN")
        return mean(array)

In [7]:
def getScalabrinoReadability(sample, commit):
    checkoutTo(sample, commit)
    r = ScalabrinoReadability(sample.split("/")[0]+"\\"+sample.split("/")[1])
    readability = r.getReadability()
    del r
    return readability

In [8]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [20]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")

In [21]:
samples["full_name"] = samples["framework"] + "/" + samples["path"]

In [22]:
samples = samples["full_name"].unique().tolist()

In [23]:
for sample in samples:
    print(sample)
    file_address = f"metrics/{sample}.csv"
    metrics = pd.read_csv(file_address)
    metrics["Scalabrino Readability"] = 0
    last_commit_result = ("0",0)
    printStatus(samples.index(sample), len(samples))
    for index in range(0, len(metrics)):
        current_commit = metrics.iloc[index]["commitSha"]
        if not current_commit == last_commit_result[0]:
            last_commit_result = (current_commit, getBuseReadability(sample, current_commit))
        metrics.iloc[index, len(metrics.columns.values) - 1] = last_commit_result[1]
    metrics.to_csv(file_address, index=False)

googlesamples/io2014-codelabs
0.0% Completed samples
googlesamples/android-play-publisher-api
0.53475935828877% Completed samples
googlesamples/androidtv-sample-inputs
1.06951871657754% Completed samples


KeyboardInterrupt: 